## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-11-51-41 +0000,bbc,Serious questions for Tories over Afghan data ...,https://www.bbc.com/news/articles/c98w2e9leywo
1,2025-07-16-11-41-40 +0000,bbc,Suspect who fatally shot Minnesota lawmakers r...,https://www.bbc.com/news/articles/cx2ljnyn451o
2,2025-07-16-11-40-29 +0000,bbc,Defence secretary 'unable to say' if anyone ki...,https://www.bbc.com/news/articles/c1k8yvj89kyo
3,2025-07-16-11-35-46 +0000,bbc,At least 20 killed in crush at US-backed GHF a...,https://www.bbc.com/news/articles/cg4rwrkdlzxo
4,2025-07-16-11-34-22 +0000,bbc,Barclays fined millions over financial crime r...,https://www.bbc.com/news/articles/cpwqeyj1d15o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,9
151,ukraine,6
223,new,6
4,data,5
144,tariffs,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
40,2025-07-15-18-44-15 +0000,nyt,Rising Inflation Underscores Risks in Trump’s ...,https://www.nytimes.com/2025/07/15/us/politics...,26
25,2025-07-16-01-22-14 +0000,nyt,Republicans in Congress Shift to Backing Ukrai...,https://www.nytimes.com/2025/07/15/us/politics...,22
24,2025-07-16-01-33-21 +0000,nyt,Trump’s Shift on Ukraine Is Good News for Euro...,https://www.nytimes.com/2025/07/15/world/europ...,21
35,2025-07-15-20-10-21 +0000,bbc,Three key questions after Afghan data breach s...,https://www.bbc.com/news/articles/c9w12kdg5zko,21
53,2025-07-15-14-57-21 +0000,bbc,Two dead after flash flooding in New Jersey an...,https://www.bbc.com/news/articles/c0j42xy47q7o,20


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
40,26,2025-07-15-18-44-15 +0000,nyt,Rising Inflation Underscores Risks in Trump’s ...,https://www.nytimes.com/2025/07/15/us/politics...
35,21,2025-07-15-20-10-21 +0000,bbc,Three key questions after Afghan data breach s...,https://www.bbc.com/news/articles/c9w12kdg5zko
44,16,2025-07-15-17-49-34 +0000,bbc,Carney says US-Canada trade deal likely to inc...,https://www.bbc.com/news/articles/c0q82qqxegno
50,16,2025-07-15-15-08-21 +0000,bbc,From military ruler to democrat - ex-Nigeria P...,https://www.bbc.com/news/articles/czxw2wn5v52o
3,13,2025-07-16-11-35-46 +0000,bbc,At least 20 killed in crush at US-backed GHF a...,https://www.bbc.com/news/articles/cg4rwrkdlzxo
27,13,2025-07-15-23-20-31 +0000,nyt,State Dept. Layoffs Hit Russia and Ukraine Ana...,https://www.nytimes.com/2025/07/15/us/politics...
37,11,2025-07-15-20-07-27 +0000,nyt,Police Arrest Far-Right Leader After Anti-Immi...,https://www.nytimes.com/2025/07/15/world/europ...
43,11,2025-07-15-18-27-07 +0000,nyt,Canada Wildfire Smoke Triggers Air Quality Ale...,https://www.nytimes.com/2025/07/14/world/canad...
9,10,2025-07-16-10-31-10 +0000,bbc,Trip drink ad banned over claim it makes you calm,https://www.bbc.com/news/articles/c14e2d2ednpo
19,10,2025-07-16-06-32-47 +0000,bbc,'No respite for consumer' as food prices remai...,https://www.bbc.com/news/articles/c0q82xqv0y8o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
